# **Section 3: Form a portfolio, data selection, and necessary transformation**

In [13]:
import pandas as pd
import yfinance as yf
import numpy as np
from statsmodels.tsa.stattools import adfuller
from portfolio import *

## Part 1 - Creating portfolio

In [551]:
tickers = [
    'AAPL', 'MSFT',                                   # Information Technology
    'BIIB', 'JNJ', 'LLY', 'MRK', 'PFE',               # Health Care
    'AMZN', 'NKE',                                    # Consumer Discretionary
    'JPM', 'BAC', 'C', 'MS',                          # Financials
    'GOOGL',                                          # Communication Services
    'HON', 'UNP',                                     # Industrials
    'PG', 'KO', 'WMT', 'CL', 'TSN',                   # Consumer Staples
    'XOM', 'CVX',                                     # Energy
    'NEE',                                            # Utilities
    'PLD', 'AMT',                                     # Real Estate
    'LIN'                                             # Materials
]

equal_weight = 1/len(tickers)
weights = {ticker:equal_weight for ticker in tickers}
prices = yf.download(tickers, start='1976-04-01', end='2023-10-01', interval='1mo', progress=False)['Adj Close']

## Part 2 - Clean data

In [552]:
prices.index = pd.to_datetime(prices.index)
prices.index = prices.index.strftime('%Y-%m')
prices.dropna(inplace=True)
prices.head()

Ticker,AAPL,AMT,AMZN,BAC,BIIB,C,CL,CVX,GOOGL,HON,...,MSFT,NEE,NKE,PFE,PG,PLD,TSN,UNP,WMT,XOM
Date,,,,,,,,,,,,,,,,,,,,,
2004-09,0.583704,11.720683,2.0430,27.297621,61.169998,278.599152,14.123345,25.307062,3.235232,21.670815,...,17.250334,4.690566,7.723265,12.824572,30.943264,18.852247,11.653690,9.766553,11.790461,24.749004
2004-10,0.789319,13.125636,1.7065,28.502569,58.160000,280.177856,13.948284,25.033421,4.758987,20.353401,...,17.449980,4.730387,7.979585,12.133054,29.262318,19.096680,10.547972,10.549776,11.950030,25.204758
2004-11,1.009997,13.843387,1.9840,29.444391,58.680000,285.172699,14.453038,25.759979,4.542805,21.350527,...,16.726267,4.828566,8.308353,11.638513,30.722250,20.580601,11.922843,10.628515,11.537802,26.244291
2004-12,0.970079,14.049550,2.2145,29.902548,66.610001,307.030731,16.077797,24.962437,4.812658,21.512030,...,18.579523,5.180114,8.900119,11.337539,31.641399,20.807276,13.417186,11.266831,11.706240,26.391447
2005-01,1.158371,13.835752,2.1610,29.797808,64.959999,312.574768,16.511477,25.860916,4.883303,21.858313,...,18.273573,5.311088,8.513958,10.186497,30.578648,19.383286,12.520276,10.032343,11.641442,26.566486


## Part 4 - Stationarity tests on MEVs

In [553]:
MEVs = pd.read_csv('2024-Table_2A_Historic_Domestic.csv')
quarter_to_month = {'Q1': '03', 'Q2': '06', 'Q3': '09', 'Q4': '12'}

def convert_to_yyyy_mm(quarter_str):
    year, quarter = quarter_str.split()
    month = quarter_to_month[quarter]
    return f"{year}-{month}"
    
MEVs['Date'] = MEVs['Date'].apply(convert_to_yyyy_mm)
MEVs.set_index('Date', inplace=True)
MEVs.drop(columns=['Scenario Name'], inplace=True)

MEVs.head()

,Real GDP growth,Nominal GDP growth,Real disposable income growth,Nominal disposable income growth,Unemployment rate,CPI inflation rate,3-month Treasury rate,5-year Treasury yield,10-year Treasury yield,BBB corporate yield,Mortgage rate,Prime rate,Dow Jones Total Stock Market Index (Level),House Price Index (Level),Commercial Real Estate Price Index (Level),Market Volatility Index (Level)
Date,,,,,,,,,,,,,,,,
1976-03,9.3,14.0,5.0,9.6,7.7,4.7,4.9,7.4,7.6,NaN,8.9,6.8,NaN,22.9,50.9,NaN
1976-06,3.0,7.2,2.3,5.8,7.6,3.6,5.2,7.4,7.6,NaN,8.8,6.9,NaN,23.6,51.8,NaN
1976-09,2.2,7.6,3.2,9.6,7.7,6.5,5.2,7.3,7.6,NaN,9.0,7.1,NaN,24.2,52.6,NaN
1976-12,2.9,10.5,2.6,9.2,7.8,5.9,4.7,6.5,7.1,NaN,8.8,6.5,NaN,25.2,53.4,NaN
1977-03,4.8,11.7,0.9,8.4,7.5,7.5,4.6,6.8,7.2,NaN,8.7,6.3,NaN,26.2,55.0,NaN


In [554]:
def make_stationary(series, significance_level=0.05):
    """
    Differentiates a pandas Series until it becomes stationary based on the Augmented Dickey-Fuller test.
    
    Parameters:
    - series: pd.Series - The time series data to test for stationarity.
    - significance_level: float - The p-value threshold to consider the series stationary (default is 0.05).
    
    Returns:
    - num_diffs: int - The number of differences needed to achieve stationarity.
    """
    num_diffs = 0
    diff_series = series.copy()
    
    while True:

        adf_test = adfuller(diff_series.dropna())
        p_value = adf_test[1]
        
        if p_value < significance_level:
            return num_diffs, diff_series
        
        diff_series = diff_series.diff().dropna()
        num_diffs += 1

In [555]:
diffs_needed = pd.DataFrame(index=MEVs.columns, columns=['Differences'])
for MEV in MEVs.columns:
    stationary = make_stationary(MEVs[MEV])
    diffs_needed.loc[MEV] = stationary[0]
    if stationary[0] != 0:
        MEVs[MEV] = stationary[1]
MEVs = MEVs.dropna()
MEVs.head()

,Real GDP growth,Nominal GDP growth,Real disposable income growth,Nominal disposable income growth,Unemployment rate,CPI inflation rate,3-month Treasury rate,5-year Treasury yield,10-year Treasury yield,BBB corporate yield,Mortgage rate,Prime rate,Dow Jones Total Stock Market Index (Level),House Price Index (Level),Commercial Real Estate Price Index (Level),Market Volatility Index (Level)
Date,,,,,,,,,,,,,,,,
1990-03,4.4,9.0,3.3,2.6,5.3,3.0,0.1,0.5,0.5,0.6,0.3,-0.5,-146.4,1.1,-1.0,27.3
1990-06,1.5,6.1,3.0,-2.6,5.3,-3.1,-0.1,0.2,0.3,0.3,0.2,0.0,150.9,0.0,-0.9,24.2
1990-09,0.3,3.7,0.1,-1.6,5.7,3.1,-0.2,-0.2,0.0,-0.1,-0.2,0.0,-545.1,-0.1,-0.5,36.5
1990-12,-3.6,-0.7,-3.2,-3.1,6.1,-0.1,-0.5,-0.4,-0.3,0.3,-0.1,0.0,222.1,-0.4,-0.4,34.0
1991-03,-1.9,2.0,1.2,1.3,6.6,-4.0,-1.0,-0.4,-0.3,-0.5,-0.5,-0.8,482.3,-0.6,-1.0,36.2


In [556]:
diffs_needed[diffs_needed['Differences']!=0]

,Differences
Nominal disposable income growth,1
CPI inflation rate,1
3-month Treasury rate,1
5-year Treasury yield,1
10-year Treasury yield,1
BBB corporate yield,1
Mortgage rate,1
Prime rate,1
Dow Jones Total Stock Market Index (Level),1
House Price Index (Level),1


## Part 5 - Summary Statistics

### For the stock returns:

In [557]:
prices.describe().T

,count,mean,std,min,25%,50%,75%,max
Ticker,,,,,,,,
AAPL,229.0,41.089456,51.693560,0.583704,5.583982,19.840931,45.121765,194.971771
AMT,229.0,93.712152,73.646325,11.720683,32.260506,65.349602,142.240707,267.634766
AMZN,229.0,45.629840,53.076733,1.344500,4.528000,16.122000,85.936501,175.353500
BAC,229.0,21.329719,10.299661,3.136746,12.113324,21.555002,29.444391,44.192055
BIIB,229.0,188.913712,115.398784,34.450001,57.389999,219.289993,289.730011,422.239990
C,229.0,94.389897,107.034425,11.438826,35.938503,43.804787,61.727081,384.071869
CL,229.0,45.405716,19.576115,13.948284,26.109671,50.561008,61.057644,79.470909
CVX,229.0,70.391147,32.970923,24.891180,43.791649,69.687386,85.466713,169.508759
GOOGL,229.0,40.853445,37.465174,3.235232,12.862295,26.935305,56.320042,147.680328


### The stock prices range from October of 2004 up to October of 2023 on a monthly basis.

### For the MEVs:

In [558]:
MEVs.describe().T

,count,mean,std,min,25%,50%,75%,max
Real GDP growth,136.0,2.541912,4.532672,-28.0,1.475,2.55,4.025,34.8
Nominal GDP growth,136.0,4.873529,5.098456,-29.2,3.600,5.00,6.500,39.7
Real disposable income growth,136.0,2.811029,7.538795,-27.6,1.200,2.75,4.225,56.0
Nominal disposable income growth,136.0,-0.017647,12.432013,-86.6,-1.925,0.20,2.000,69.3
Unemployment rate,136.0,5.758088,1.742946,3.5,4.475,5.40,6.700,13.0
CPI inflation rate,136.0,-0.009559,2.423462,-15.2,-1.200,0.00,1.025,8.4
3-month Treasury rate,136.0,-0.017647,0.426334,-1.3,-0.100,0.00,0.200,1.6
5-year Treasury yield,136.0,-0.025735,0.415977,-1.1,-0.300,0.00,0.200,1.2
10-year Treasury yield,136.0,-0.025735,0.368178,-0.9,-0.300,0.00,0.200,1.0
BBB corporate yield,136.0,-0.026471,0.484613,-1.6,-0.300,0.00,0.225,2.6


### The MEVs range from Q1 of 1990 to Q3 of 2023. Chances are in the data analysis where the MEVs are used, the years that are in this dataset preceding those in the stock returns dataset will be dropped.

## **Section 4: Stress Testing using Fama-French three-factor model**

## Part 1 - Picking a subset of MEVs

### We're going to try and capture the big economic picture at each time step while avoiding redundancy. In order to do this, we chose to use the CPI, real GDP growth, real disposable income growth, 3 month, 5 year, and 10 year rates.

In [559]:
MEVs = MEVs[['Real GDP growth', 
             'Real disposable income growth', 
             'CPI inflation rate', 
             '3-month Treasury rate',
             '5-year Treasury yield',
             '10-year Treasury yield',
             'Market Volatility Index (Level)'
            ]]

## Part 2 - Report the results from the Fama-French three factor model

In [560]:
#portfolio returns

returns = prices.pct_change()[1:]
returns['portfolio'] = sum(returns[stock] * weights[stock] for stock, weight in weights.items()) # Setting the portfolio returns based on weights

In [561]:
FF_factors = pd.read_excel('wrds_data.xlsx', sheet_name='returns', index_col=0)

c:\Users\justi\anaconda3\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


In [562]:
FF_factors = pd.read_excel('wrds_data.xlsx', sheet_name='returns', index_col=0)
FF_factors = FF_factors[['FAMA-FRENCH MARKET FACTOR', 'FAMA-FRENCH SIZE FACTOR (SMB)', 'FAMA-FRENCH VALUE FACTOR (HML)', 'MOMENTUM FACTOR']]
FF_factors = FF_factors.rename(columns={'FAMA-FRENCH MARKET FACTOR': 'MKT', 'FAMA-FRENCH SIZE FACTOR (SMB)': 'SMB', 'FAMA-FRENCH VALUE FACTOR (HML)': 'HML', 'MOMENTUM FACTOR': 'UMD'})
FF_factors.head()


c:\Users\justi\anaconda3\Lib\site-packages\openpyxl\worksheet\header_footer.py:48: UserWarning: Cannot parse header or footer so it will be ignored
  warn("""Cannot parse header or footer so it will be ignored""")


,MKT,SMB,HML,UMD
Date,,,,
1963-01,0.0493,0.0308,0.0221,-0.0211
1963-02,-0.0238,0.0048,0.0218,0.0253
1963-03,0.0308,-0.0259,0.0206,0.0162
1963-04,0.0451,-0.0134,0.0100,-0.0009
1963-05,0.0176,0.0113,0.0254,0.0033


In [563]:
FF_model = FF_factors.copy()
FF_model['portfolio'] = returns['portfolio']
FF_model = FF_model.dropna()
FF_model

,MKT,SMB,HML,UMD,portfolio
Date,,,,,
2004-10,0.0143,0.0015,-0.0022,-0.0138,0.022146
2004-11,0.0454,0.0374,0.0141,0.0316,0.029243
2004-12,0.0343,-0.0003,-0.0022,-0.0287,0.051720
2005-01,-0.0276,-0.0172,0.0206,0.0305,-0.013803
2005-02,0.0189,-0.0057,0.0154,0.0337,0.014125
...,...,...,...,...,...
2023-05,0.0035,0.0160,-0.0774,-0.0065,-0.034575
2023-06,0.0647,0.0155,-0.0020,-0.0235,0.043780
2023-07,0.0321,0.0205,0.0411,-0.0405,0.022336


In [ ]:
FF_portfolio = get_ols_metrics(FF_model[['MKT', 'SMB', 'HML']], FF_model['portfolio'])
FF_portfolio

,alpha,MKT,SMB,HML,r-squared,Info Ratio
portfolio,0.004894,0.921658,-0.26738,0.105062,0.904096,0.373125


## The Fama-French model for our portfolio is given by: 
$$
\Huge
E[r_i] = 0.92(MKT - r_f) - 0.3(SMB) + 0.1(HML) - 0.02(UMD) + 0.01 
$$

## Part 3 - Identify the impact of our chosen MEVs on Fama-French factors.

### Note: Here we merged the dataframes of the FF factors with the MEVs dataframe, inherently dropping all months that don't fall at the end of a quarter. This was to maintain uniformity and ensure an accurate result. We felt this method was better than interpolating MEV data because those numbers come straight from the Fed and it wouldn't be reliable to try and subjectively interpret gaps in data.

In [567]:
FF_factors.reset_index(inplace=True)

In [568]:
FF_factors

,Date,MKT,SMB,HML,UMD
0,1963-01,0.0493,0.0308,0.0221,-0.0211
1,1963-02,-0.0238,0.0048,0.0218,0.0253
2,1963-03,0.0308,-0.0259,0.0206,0.0162
3,1963-04,0.0451,-0.0134,0.0100,-0.0009
4,1963-05,0.0176,0.0113,0.0254,0.0033
...,...,...,...,...,...
727,2023-08,-0.0239,-0.0320,-0.0108,0.0377
728,2023-09,-0.0524,-0.0249,0.0145,0.0024
729,2023-10,-0.0318,-0.0388,0.0019,0.0168
730,2023-11,0.0883,-0.0003,0.0166,0.0276


In [569]:
# Extract quarter and aggregate the data
FF_factors['Date'] = pd.to_datetime(FF_factors['Date'])
FF_factors['Quarter'] = FF_factors['Date'].dt.to_period('Q')


# Calculate cumulative returns for each FF portfolio within each quarter
# Assuming monthly returns are given as percentages, so convert to decimals for compounding
FF_factors['MKT'] = FF_factors['MKT'] /100 + 1
FF_factors['SMB'] = FF_factors['SMB'] /100 + 1
FF_factors['HML'] = FF_factors['HML'] /100  + 1

# Aggregate cumulative returns by quarter using product to compound
quarterly_returns = FF_factors.groupby('Quarter').agg({
    'MKT': 'prod',
    'SMB': 'prod',
    'HML': 'prod'
}).reset_index()

# Convert back to percentage returns
quarterly_returns[['MKT', 'SMB', 'HML']] = (quarterly_returns[['MKT', 'SMB', 'HML']] - 1) * 100


quarterly_returns = quarterly_returns.set_index('Quarter')
quarterly_returns = quarterly_returns['1990':]
quarterly_returns


,MKT,SMB,HML
Quarter,,,
1990Q1,-0.049121,0.012498,-0.014304
1990Q2,0.039666,-0.015903,-0.083478
1990Q3,-0.181607,-0.103364,0.022501
1990Q4,0.068899,-0.043307,-0.046995
1991Q1,0.145365,0.116946,-0.035696
...,...,...,...
2022Q4,0.060256,-0.041298,0.108124
2023Q1,0.065593,0.006672,-0.136954
2023Q2,0.074306,-0.002108,-0.079898


In [570]:
MEVs.index = pd.to_datetime(MEVs.index)
MEVs = MEVs['1990':]
MEVs = MEVs[:(len(quarterly_returns))]
MEVs


,Real GDP growth,Real disposable income growth,CPI inflation rate,3-month Treasury rate,5-year Treasury yield,10-year Treasury yield,Market Volatility Index (Level)
Date,,,,,,,
1990-03-01,4.4,3.3,3.0,0.1,0.5,0.5,27.3
1990-06-01,1.5,3.0,-3.1,-0.1,0.2,0.3,24.2
1990-09-01,0.3,0.1,3.1,-0.2,-0.2,0.0,36.5
1990-12-01,-3.6,-3.2,-0.1,-0.5,-0.4,-0.3,34.0
1991-03-01,-1.9,1.2,-4.0,-1.0,-0.4,-0.3,36.2
...,...,...,...,...,...,...,...
2022-12-01,2.6,2.2,-1.3,1.3,0.8,0.7,33.6
2023-03-01,2.2,10.8,-0.4,0.6,-0.3,-0.2,26.5
2023-06-01,2.1,3.3,-1.1,0.5,-0.1,0.0,20.1


In [571]:
quarterly_returns.index = MEVs.index

# Merge the dataframes on the index
FF_MEVs = MEVs.merge(quarterly_returns, left_index=True, right_index=True, how='inner')
FF_MEVs.head()

,Real GDP growth,Real disposable income growth,CPI inflation rate,3-month Treasury rate,5-year Treasury yield,10-year Treasury yield,Market Volatility Index (Level),MKT,SMB,HML
Date,,,,,,,,,,
1990-03-01,4.4,3.3,3.0,0.1,0.5,0.5,27.3,-0.049121,0.012498,-0.014304
1990-06-01,1.5,3.0,-3.1,-0.1,0.2,0.3,24.2,0.039666,-0.015903,-0.083478
1990-09-01,0.3,0.1,3.1,-0.2,-0.2,0.0,36.5,-0.181607,-0.103364,0.022501
1990-12-01,-3.6,-3.2,-0.1,-0.5,-0.4,-0.3,34.0,0.068899,-0.043307,-0.046995
1991-03-01,-1.9,1.2,-4.0,-1.0,-0.4,-0.3,36.2,0.145365,0.116946,-0.035696


In [572]:
import statsmodels.api as sm

reg_models = []
factors = ['MKT', 'SMB', 'HML']
for factor in factors:
    model = sm.OLS(FF_MEVs[factor], sm.add_constant(FF_MEVs[['Real GDP growth', 'Real disposable income growth', 'CPI inflation rate', '3-month Treasury rate', '5-year Treasury yield', '10-year Treasury yield', 'Market Volatility Index (Level)']])).fit()
    reg_models.append(model)

reg_models

for reg_model in reg_models:
    print(reg_model.summary())




                            OLS Regression Results                            
Dep. Variable:                    MKT   R-squared:                       0.155
Model:                            OLS   Adj. R-squared:                  0.109
Method:                 Least Squares   F-statistic:                     3.350
Date:                Sun, 17 Nov 2024   Prob (F-statistic):            0.00259
Time:                        20:09:24   Log-Likelihood:                 155.45
No. Observations:                 136   AIC:                            -294.9
Df Residuals:                     128   BIC:                            -271.6
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
const     

### This dataframe shows the coefficients of the linear regressions:
$$
\Huge
FF_i = \alpha_i + \beta_{1_i}MEV_1 + \beta_2MEV_2 + \ldots + \beta_nMEV_n
$$

## Part 4 - Investigating the impact of the MEVs on Fama-French factors during *stressed times*

### We are picking all date ranges of stressed times after 1976, where our data for FF factors and MEVs starts. This gives a full picture on the effect of the MEVs on the FF factors throughout history of the past ~5 decades, whereas a smaller sample size wouldn't accurately describe the effects.

In [573]:
#df = FF_factors.merge(MEVs, left_index=True, right_index=True, how='inner')
stressed_date_ranges = [('1990-09', '1991-03'),
                        ('2001-03', '2001-12'),
                        ('2007-12', '2009-06'),
                        ('2019-12', '2020-06')
                       ]

stressed_data = pd.concat(
    [FF_MEVs.loc[start:end] for start, end in stressed_date_ranges]
)

stressed_data

stressed_reg_models = []
factors = ['MKT', 'SMB', 'HML']
for factor in factors:
    model = sm.OLS(stressed_data[factor], sm.add_constant(stressed_data[['Real GDP growth', 'Real disposable income growth', 'CPI inflation rate', '3-month Treasury rate', '5-year Treasury yield', '10-year Treasury yield', 'Market Volatility Index (Level)']])).fit()
    stressed_reg_models.append(model)

stressed_reg_models

for reg_model in stressed_reg_models:
    print(reg_model.summary())

                            OLS Regression Results                            
Dep. Variable:                    MKT   R-squared:                       0.710
Model:                            OLS   Adj. R-squared:                  0.485
Method:                 Least Squares   F-statistic:                     3.153
Date:                Sun, 17 Nov 2024   Prob (F-statistic):             0.0562
Time:                        20:09:24   Log-Likelihood:                 19.988
No. Observations:                  17   AIC:                            -23.98
Df Residuals:                       9   BIC:                            -17.31
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
                                      coef    std err          t      P>|t|      [0.025      0.975]
---------------------------------------------------------------------------------------------------
const     

c:\Users\justi\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:1806: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=17
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
c:\Users\justi\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:1806: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=17
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "
c:\Users\justi\anaconda3\Lib\site-packages\scipy\stats\_stats_py.py:1806: UserWarning: kurtosistest only valid for n>=20 ... continuing anyway, n=17
  warnings.warn("kurtosistest only valid for n>=20 ... continuing "


In [574]:
stressed_MEVs_effects_on_factors = pd.DataFrame(index=FF_model.columns[:-2], columns=MEVs.columns)
regressors = stressed_data[MEVs.columns]
for factor in FF_model.columns[:-2]:
    reg = get_ols_metrics(regressors, stressed_data[factor])
    for MEV in reg.columns[1:-2]:
        stressed_MEVs_effects_on_factors.loc[factor, MEV] = reg[MEV].values[0]
stressed_MEVs_effects_on_factors

,Real GDP growth,Real disposable income growth,CPI inflation rate,3-month Treasury rate,5-year Treasury yield,10-year Treasury yield,Market Volatility Index (Level)
MKT,-0.031836,-0.008634,0.005324,-0.191436,0.26366,0.078913,-0.005927
SMB,-0.014483,-0.005786,0.0036,-0.182453,0.206125,-0.019385,-0.00137
HML,-0.003538,-0.001055,-0.001456,0.07248,-0.254621,0.283381,-0.003722


### This table similarly shows the coefficients of the MEVs on FF factors.

## Part 5 - Projecting the performance of our portfolio

In [575]:
# Read the CSV file and set 'Date' as the index
MEVs_severe = pd.read_csv('2024-Table_4A_Supervisory_Severely_Adverse_Domestic.csv')

MEVs_severe['Date'] = MEVs_severe['Date'].apply(convert_to_yyyy_mm)

# Drop the 'Scenario Name' column
MEVs_severe.drop(columns=['Scenario Name'], inplace=True)
MEVs_severe.set_index('Date', inplace=True)

# Display the DataFrame
MEVs_severe = MEVs_severe[MEVs.columns]

MEVs_severe

,Real GDP growth,Real disposable income growth,CPI inflation rate,3-month Treasury rate,5-year Treasury yield,10-year Treasury yield,Market Volatility Index (Level)
Date,,,,,,,
2024-03,-11.6,-7.8,2.3,2.1,0.4,1.1,65.0
2024-06,-6.7,-4.0,1.5,0.2,0.3,0.8,70.0
2024-09,-8.0,-4.2,1.3,0.1,0.4,0.8,61.4
2024-12,-5.9,-2.9,1.3,0.1,0.5,0.8,54.5
2025-03,-1.8,-0.1,1.4,0.1,0.5,0.9,49.1
2025-06,0.6,1.2,1.4,0.1,0.6,1.0,44.8
2025-09,0.9,1.7,1.4,0.1,0.7,1.1,41.5
2025-12,6.5,5.3,1.5,0.1,0.7,1.2,38.8
2026-03,6.1,5.6,1.5,0.1,0.8,1.3,36.6


In [586]:
import pandas as pd
import statsmodels.api as sm

def predict_FF_factors(MEVs, model):
    """
    Predicts the FF factors using the provided MEVs and trained model.

    Parameters:
    MEVs (pd.DataFrame): DataFrame containing the macroeconomic variables.
    model (sm.OLS): Trained OLS regression model.

    Returns:
    pd.DataFrame: DataFrame containing the predicted FF factors.
    """
    # Ensure MEVs is a DataFrame
    if not isinstance(MEVs, pd.DataFrame):
        raise ValueError("MEVs should be a pandas DataFrame")

    # Add a constant to the MEVs for the intercept term
    MEVs_with_const = sm.add_constant(MEVs)

    # Predict the FF factors
    predicted_FF_factors = model.predict(MEVs_with_const)

    # Convert the predictions to a DataFrame
    predicted_FF_factors_df = pd.DataFrame(predicted_FF_factors, index=MEVs.index, columns=['Predicted_FF_Factor'])

    return predicted_FF_factors_df

# Example usage:
# Assuming `MEVs_severe` is your DataFrame with MEVs and `FF_factors` is your DataFrame with FF factors
# Add a constant to the MEVs for the intercept term
MEVs_with_const = sm.add_constant(MEVs_severe)

# Train the model
model = sm.OLS(FF_factors[0], MEVs_with_const).fit()

# Predict the FF factors
predicted_FF_factors = predict_FF_factors(MEVs_severe, model)

# Print the predicted FF factors
print(predicted_FF_factors)

KeyError: 0

In [587]:
FF_factors

,Date,MKT,SMB,HML,UMD,Quarter
0,1963-01-01,1.000493,1.000308,1.000221,-0.0211,1963Q1
1,1963-02-01,0.999762,1.000048,1.000218,0.0253,1963Q1
2,1963-03-01,1.000308,0.999741,1.000206,0.0162,1963Q1
3,1963-04-01,1.000451,0.999866,1.000100,-0.0009,1963Q2
4,1963-05-01,1.000176,1.000113,1.000254,0.0033,1963Q2
...,...,...,...,...,...,...
727,2023-08-01,0.999761,0.999680,0.999892,0.0377,2023Q3
728,2023-09-01,0.999476,0.999751,1.000145,0.0024,2023Q3
729,2023-10-01,0.999682,0.999612,1.000019,0.0168,2023Q4
730,2023-11-01,1.000883,0.999997,1.000166,0.0276,2023Q4


In [576]:
# Handle stationarity similar to before
diffs_needed = pd.DataFrame(index=MEVs_severe.columns, columns=['Differences'])

for MEV in MEVs_severe.columns:
    series = MEVs_severe[MEV]
    if series.nunique() == 1:
        diffs_needed.loc[MEV] = 0
        continue
    
    stationary = make_stationary(series)
    diffs_needed.loc[MEV] = stationary[0]
    if stationary[0] != 0:
        MEVs_severe[MEV] = stationary[1]

#MEVs_severe = MEVs_severe.dropna()

MEVs_severe

c:\Users\justi\anaconda3\Lib\site-packages\statsmodels\regression\linear_model.py:956: RuntimeWarning: divide by zero encountered in log
  llf = -nobs2*np.log(2*np.pi) - nobs2*np.log(ssr / nobs) - nobs2


,Real GDP growth,Real disposable income growth,CPI inflation rate,3-month Treasury rate,5-year Treasury yield,10-year Treasury yield,Market Volatility Index (Level)
Date,,,,,,,
2024-03,NaN,NaN,NaN,2.1,NaN,NaN,65.0
2024-06,NaN,3.8,-0.8,0.2,-0.1,NaN,70.0
2024-09,-6.200000e+00,-0.2,-0.2,0.1,0.1,NaN,61.4
2024-12,3.400000e+00,1.3,0.0,0.1,0.1,NaN,54.5
2025-03,2.000000e+00,2.8,0.1,0.1,0.0,4.000000e-01,49.1
2025-06,-1.700000e+00,1.3,0.0,0.1,0.1,-2.000000e-01,44.8
2025-09,-2.100000e+00,0.5,0.0,0.1,0.1,1.000000e-01,41.5
2025-12,5.300000e+00,3.6,0.1,0.1,0.0,-4.440892e-16,38.8
2026-03,-6.000000e+00,0.3,0.0,0.1,0.1,7.771561e-16,36.6


In [577]:
MEVs_severe.head()

,Real GDP growth,Real disposable income growth,CPI inflation rate,3-month Treasury rate,5-year Treasury yield,10-year Treasury yield,Market Volatility Index (Level)
Date,,,,,,,
2024-03,NaN,NaN,NaN,2.1,NaN,NaN,65.0
2024-06,NaN,3.8,-0.8,0.2,-0.1,NaN,70.0
2024-09,-6.2,-0.2,-0.2,0.1,0.1,NaN,61.4
2024-12,3.4,1.3,0.0,0.1,0.1,NaN,54.5
2025-03,2.0,2.8,0.1,0.1,0.0,0.4,49.1


In [578]:
alltime_MEVs_effects_on_factors

,Real GDP growth,Real disposable income growth,CPI inflation rate,3-month Treasury rate,5-year Treasury yield,10-year Treasury yield,Market Volatility Index (Level)
Date,NaN,NaN,NaN,NaN,NaN,NaN,NaN
MKT,NaN,NaN,NaN,NaN,NaN,NaN,NaN
SMB,NaN,NaN,NaN,NaN,NaN,NaN,NaN
HML,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UMD,NaN,NaN,NaN,NaN,NaN,NaN,NaN
portfolio,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [579]:
# TODO: run the regression using all time MEV coefficients against severe MEV values to get MKT, SMB, HML, UMD values 
# and then plug those values into the regression equation for earlier at each time step to get portfolio return